```load data```

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import warnings

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier
from scipy.stats import rankdata
from catboost import CatBoostClassifier


train_df = pd.read_csv("/kaggle/input/playground-series-s6e2/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")

print(train_df.shape, test_df.shape)

In [ ]:
warnings.filterwarnings("ignore")

try:
    import torch
    USE_GPU = torch.cuda.is_available()
except Exception:
    USE_GPU = False

print(f"Hardware: {'GPU' if USE_GPU else 'CPU'}")

```Separate Features & Target```

In [ ]:
ID_COL = "id"
TARGET_COL = "Heart Disease"   
train_df.drop('id', axis=1, inplace=True) # id is not used for model training and prediction
X = train_df.drop(columns=[TARGET_COL]).copy()

y_raw = train_df[TARGET_COL].copy()

# Convert target to 0/1
# Absence -> 0, Presence -> 1
y = y_raw.map({"Absence": 0, "Presence": 1}).astype(int)

X_test = test_df

# Drop ID column
if ID_COL in X.columns:
    X = X.drop(columns=[ID_COL])
if ID_COL in X_test.columns:
    X_test = X_test.drop(columns=[ID_COL])

print("X shape:", X.shape)
print("X_test shape:", X_test.shape)
print("Target distribution:")
print(y.value_counts())

```Identify Column```

In [ ]:
cat_cols = [
    "Sex",
    "Chest pain type",
    "FBS over 120",
    "EKG results",
    "Exercise angina",
    "Slope of ST",
    "Number of vessels fluro",
    "Thallium"
]

num_cols = [
    "Age",
    "BP",
    "Cholesterol",
    "Max HR",
    "ST depression"
]
extra_cat_cols = ["Chol_bin", "Age_bin", "ST_bin"]

```Frequency Encoding```

In [ ]:
def make_freq_features(df_train, df_test, cols):
    df_all = pd.concat([df_train[cols], df_test[cols]], axis=0, ignore_index=True)
    tr_out = pd.DataFrame(index=df_train.index)
    te_out = pd.DataFrame(index=df_test.index)

    for c in cols:
        freqs = df_all[c].value_counts(normalize=True)
        tr_out[f"{c}_freq"] = df_train[c].map(freqs).astype(np.float32)
        te_out[f"{c}_freq"] = df_test[c].map(freqs).astype(np.float32)

    return tr_out, te_out

train_freq, test_freq = make_freq_features(train_df, test_df, cat_cols + num_cols)

```K-fold target mean encoding```

In [ ]:
def make_target_mean_features(df_train, y, df_test, cols, n_splits=5, seed=42, alpha=50):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    tr_out = pd.DataFrame(index=df_train.index)
    te_out = pd.DataFrame(index=df_test.index)

    global_mean = float(y.mean())

    for c in cols:
        tr_feat = pd.Series(index=df_train.index, dtype=np.float32)

        for tr_idx, va_idx in skf.split(df_train, y):
            tr_part = df_train.iloc[tr_idx]
            y_part = y.iloc[tr_idx]

            stats = (
                pd.DataFrame({c: tr_part[c].values, "y": y_part.values})
                .groupby(c)["y"]
                .agg(["mean", "count"])
            )

            smooth = (stats["mean"] * stats["count"] + global_mean * alpha) / (stats["count"] + alpha)
            tr_feat.iloc[va_idx] = df_train.iloc[va_idx][c].map(smooth)

        full_stats = (
            pd.DataFrame({c: df_train[c].values, "y": y.values})
            .groupby(c)["y"]
            .agg(["mean", "count"])
        )

        full_smooth = (full_stats["mean"] * full_stats["count"] + global_mean * alpha) / (full_stats["count"] + alpha)

        tr_out[f"{c}_te"] = tr_feat.fillna(global_mean).astype(np.float32)
        te_out[f"{c}_te"] = df_test[c].map(full_smooth).fillna(global_mean).astype(np.float32)

    return tr_out, te_out

train_te, test_te = make_target_mean_features(train_df, y, test_df, cat_cols, n_splits=5, seed=42, alpha=50)


```Build train-test metrics```

In [ ]:
X = pd.concat(
    [train_df[cat_cols + num_cols], train_freq, train_te],
    axis=1
)

X_test = pd.concat(
    [test_df[cat_cols + num_cols], test_freq, test_te],
    axis=1
)


```Feature Engineering```

In [ ]:
# STEP 1: Age × Max HR
X["Age_x_MaxHR"] = X["Age"] * X["Max HR"]
X_test["Age_x_MaxHR"] = X_test["Age"] * X_test["Max HR"]

# STEP 2: ST depression × Exercise angina
X["ST_x_Angina"] = X["ST depression"] * X["Exercise angina"]
X_test["ST_x_Angina"] = X_test["ST depression"] * X_test["Exercise angina"]

# STEP 3: High blood pressure flag (BP ≥ 140)
X["High_BP"] = (X["BP"] >= 140).astype(int)
X_test["High_BP"] = (X_test["BP"] >= 140).astype(int)

# STEP 4: Cholesterol risk bin (0=normal, 1=borderline, 2=high)
X["Chol_bin"] = pd.cut(X["Cholesterol"], bins=[0, 200, 240, 2000], labels=[0, 1, 2]).astype(int)
X_test["Chol_bin"] = pd.cut(X_test["Cholesterol"], bins=[0, 200, 240, 2000], labels=[0, 1, 2]).astype(int)

# STEP 5: Age risk bucket
X["Age_bin"] = pd.cut(
    X["Age"],
    bins=[0, 40, 50, 60, 70, 120],
    labels=[0, 1, 2, 3, 4]
).astype(int)

X_test["Age_bin"] = pd.cut(
    X_test["Age"],
    bins=[0, 40, 50, 60, 70, 120],
    labels=[0, 1, 2, 3, 4]
).astype(int)

# STEP 6: Low maximum heart rate flag
X["Low_MaxHR"] = (X["Max HR"] < 120).astype(int)
X_test["Low_MaxHR"] = (X_test["Max HR"] < 120).astype(int)

# STEP 7: ST depression severity bucket
X["ST_bin"] = pd.cut(
    X["ST depression"],
    bins=[-1, 0.5, 1.5, 10],
    labels=[0, 1, 2]
).astype(int)

X_test["ST_bin"] = pd.cut(
    X_test["ST depression"],
    bins=[-1, 0.5, 1.5, 10],
    labels=[0, 1, 2]
).astype(int)


# STEP 8: Simple risk score
X["Risk_score"] = (
    X["High_BP"]
    + X["Chol_bin"]
    + X["Exercise angina"]
    + X["Age_bin"]
)

X_test["Risk_score"] = (
    X_test["High_BP"]
    + X_test["Chol_bin"]
    + X_test["Exercise angina"]
    + X_test["Age_bin"]
)

In [ ]:
all_cat_cols = cat_cols + extra_cat_cols

cat_features = [X.columns.get_loc(c) for c in all_cat_cols]

print("X_train shape:", X.shape, "| X_test shape:", X_test.shape)

In [ ]:
# print("CatBoost categorical columns:")
# for idx in cat_features:
#     print(idx, X.columns[idx])

In [ ]:
neg = (y == 0).sum()
pos = (y == 1).sum()
scale_pos_weight = neg / pos

print("scale_pos_weight:", scale_pos_weight)


```XGBClassifier```

In [ ]:
def make_xgb(seed: int) -> XGBClassifier:
    return XGBClassifier(
        n_estimators=4000,
        learning_rate=0.03,
        
        max_depth=6,
        min_child_weight=3,
        
        subsample=0.85,
        colsample_bytree=0.85,
        
        scale_pos_weight=scale_pos_weight,
        early_stopping_rounds=150,
        
        gamma=0.1,
        reg_alpha=0.1,
        reg_lambda=1.0,
        objective="binary:logistic",
        eval_metric="error",
        
        tree_method="hist",
        random_state=seed,
        n_jobs=-1
    )

cat_params = {
    "iterations": 8000,
    "learning_rate": 0.03,
    "depth": 6,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "auto_class_weights": "Balanced",
    "early_stopping_rounds": 200,
    "task_type": "GPU" if USE_GPU else "CPU",
    "logging_level": "Silent"
}


```Cross-validation and seeds```

In [ ]:
# skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

xgb_seeds = [42, 202, 777]
cat_seeds = [42, 202, 777, 1001, 2023]


```Train multi-seed XGBoost (OOF + test probs)```

In [ ]:
oof_xgb = np.zeros(len(train_df), dtype=np.float32)
test_xgb = np.zeros(len(test_df), dtype=np.float32)
xgb_oof_aucs = []

for s in xgb_seeds:
    oof_s = np.zeros(len(train_df), dtype=np.float32)
    test_s = np.zeros(len(test_df), dtype=np.float32)

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
        X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        X_va, y_va = X.iloc[va_idx], y.iloc[va_idx]

        model = make_xgb(seed=s + fold)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            verbose=False
        )

        va_pred = model.predict_proba(X_va)[:, 1]
        oof_s[va_idx] = va_pred

        test_s += model.predict_proba(X_test)[:, 1] / skf.n_splits

        del model
        gc.collect()

    auc_s = roc_auc_score(y, oof_s)
    xgb_oof_aucs.append(auc_s)

    oof_xgb += oof_s / len(xgb_seeds)
    test_xgb += test_s / len(xgb_seeds)

    print(f"XGB Seed {s} | OOF AUC: {auc_s:.6f}")

print("XGB Mean OOF AUC:", float(np.mean(xgb_oof_aucs)))


```Train multi-seed CatBoost (OOF + test probs)```

In [ ]:
oof_cat = np.zeros(len(train_df), dtype=np.float32)
test_cat = np.zeros(len(test_df), dtype=np.float32)
cat_oof_aucs = []

for s in cat_seeds:
    oof_s = np.zeros(len(train_df), dtype=np.float32)
    test_s = np.zeros(len(test_df), dtype=np.float32)

    for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
        X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        X_va, y_va = X.iloc[va_idx], y.iloc[va_idx]

        model = CatBoostClassifier(**cat_params, random_seed=s + fold)
        model.fit(
            X_tr, y_tr,
            eval_set=(X_va, y_va),
            cat_features=cat_features
        )

        va_pred = model.predict_proba(X_va)[:, 1]
        oof_s[va_idx] = va_pred

        test_s += model.predict_proba(X_test)[:, 1] / skf.n_splits

        del model
        gc.collect()

    auc_s = roc_auc_score(y, oof_s)
    cat_oof_aucs.append(auc_s)

    oof_cat += oof_s / len(cat_seeds)
    test_cat += test_s / len(cat_seeds)

    print(f"CAT Seed {s} | OOF AUC: {auc_s:.6f}")

print("CAT Mean OOF AUC:", float(np.mean(cat_oof_aucs)))
print("CatBoost Std  OOF AUC:", np.std(cat_oof_aucs))

```optimal bend weight```

In [ ]:
# Rank-transform OOF predictions
rank_oof_xgb = rankdata(oof_xgb)
rank_oof_cat = rankdata(oof_cat)

# Optional: scale ranks to [0, 1] (not required for AUC, but nice)
rank_oof_xgb = (rank_oof_xgb - rank_oof_xgb.min()) / (rank_oof_xgb.max() - rank_oof_xgb.min())
rank_oof_cat = (rank_oof_cat - rank_oof_cat.min()) / (rank_oof_cat.max() - rank_oof_cat.min())

weights = np.arange(0.0, 1.01, 0.05)

best = {"auc": -1.0, "w_xgb": 0.5}

for w in weights:
    oof_blend_rank = w * rank_oof_xgb + (1.0 - w) * rank_oof_cat
    auc = roc_auc_score(y, oof_blend_rank)
    if auc > best["auc"]:
        best.update({"auc": auc, "w_xgb": w})

w0 = best["w_xgb"]
fine_weights = np.arange(max(0.0, w0 - 0.10), min(1.0, w0 + 0.10) + 1e-9, 0.01)

best_fine = {"auc": -1.0, "w_xgb": w0}

for w in fine_weights:
    oof_blend_rank = w * rank_oof_xgb + (1.0 - w) * rank_oof_cat
    auc = roc_auc_score(y, oof_blend_rank)
    if auc > best_fine["auc"]:
        best_fine.update({"auc": auc, "w_xgb": w})

w_best = best_fine["w_xgb"]

print("Best blend OOF AUC (rank):", best_fine["auc"])
print("Best w_xgb:", w_best, "| w_cat:", 1.0 - w_best)


In [ ]:
rank_test_xgb = rankdata(test_xgb)
rank_test_cat = rankdata(test_cat)

rank_test_xgb = (rank_test_xgb - rank_test_xgb.min()) / (rank_test_xgb.max() - rank_test_xgb.min())
rank_test_cat = (rank_test_cat - rank_test_cat.min()) / (rank_test_cat.max() - rank_test_cat.min())

final_test_probs = w_best * rank_test_xgb + (1.0 - w_best) * rank_test_cat

```create submission```

In [ ]:
if os.path.exists("submission.csv"):
    os.remove("submission.csv")
    
submission = pd.DataFrame({
    "id": test_df["id"],
    "Heart Disease": final_test_probs  # probabilities
})
submission.to_csv("submission.csv", index=False)
print("Saved new submission.csv")